# Map Scrambling

This is an experiment to see how time and location data can be scrambled while still allowing querying based on proximity.


## Sample Data

First, lets source some open data from the City of Cape Town in order to generate sample location data sets

In [1]:
from urllib.parse import urlencode, quote_plus
import requests
import csv
import math
import os
import hashlib
import random
import pprint
import zipfile
import pygeoif.geometry
from fastkml import kml

cache_dir = os.path.join(os.getcwd(), '_cache')
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)

def make_cache_name(base_name, input_params, suffix):
    input_str = pprint.pformat(sorted([(a, getattr(input_params, a)) for a in dir(input_params) if not a.startswith('_')]))
    filename = '%s-%s.%s' % (base_name, hashlib.sha1(input_str.encode('utf-8')).hexdigest(), suffix)
    return os.path.join(cache_dir, filename)

# TODO: you need to get the City of Cape Town consent to download these files, as described on the home page URLs
sample_data_sets = {
    'taxi_routes': {'description': 'The file stores the spatial location of features (point, line, polygon) of taxi routes.',
                    'license_url': 'https://web1.capetown.gov.za/web1/OpenDataPortal/Document/GetDocument/1',
                    'home_page': 'https://web1.capetown.gov.za/web1/OpenDataPortal/DatasetDetail?DatasetName=Taxi%20routes',
                    'url': 'http://cityapps.capetown.gov.za/sites/opendatacatalog/Documents/Taxi%20routes/Taxi_Routes.kmz',
                    'skip': False,
                   },
    'fire_stations': {'description': 'Indicates the location of fire stations.',
                    'license_url': 'https://web1.capetown.gov.za/web1/OpenDataPortal/Document/GetDocument/1',
                    'home_page': 'https://web1.capetown.gov.za/web1/OpenDataPortal/DatasetDetail?DatasetName=Fire%20stations',
                    'url': 'http://cityapps.capetown.gov.za/sites/opendatacatalog/Documents/Fire%20stations/Fire%20stations%202016.kmz',
                     }
}

points = set()
routes = []
route_points = set()

# read in all the data
for data_set_name, data_set_attrs in sample_data_sets.items():
    if data_set_attrs.get('skip', False):
        continue
    url = data_set_attrs['url']
    document_name = url[url.rfind('/')+1:]
    document_format = document_name[document_name.rfind('.')+1:]
    actual_url = 'https://www.capetown.gov.za/_layouts/OpenDataPortalHandler/DownloadHandler.ashx?DocumentName=%s&DatasetDocument=%s' % (document_name, quote_plus(url))
    url_hash = hashlib.sha256(actual_url.encode('UTF-8')).hexdigest()
    cache_filename = os.path.join(cache_dir, '%s-%s' % (url_hash, document_name))
    if os.path.exists(cache_filename):
        with open(cache_filename, 'rb') as f:
            data = f.read()
    else:
        data = requests.get(actual_url).content
        with open(cache_filename, 'wb') as f:
            f.write(data)
    if document_format == 'kmz':
        with zipfile.ZipFile(cache_filename) as kmz_zip:
            with kmz_zip.open('doc.kml') as kml_file:
                data = kml_file.read()
        document_format = 'kml'
    print("Name: %s; format: %s; length: %s" % (data_set_name, document_format, len(data)))
    k = kml.KML()
    k.from_string(data)
    count_points, count_paths, count_path_points = 0, 0, 0
    for document in k.features():
        for folder in document.features():
            for placemark in folder.features():
                geometry = placemark.geometry
                if isinstance(geometry, pygeoif.geometry.Point):
                    points.add((geometry.x, geometry.y))
                    count_points += 1
                elif isinstance(geometry, pygeoif.geometry.MultiLineString):
                    # the taxi routes data has a single linestring per MultiLineString
                    for i, linestring in enumerate(geometry.geoms):
                        line_points = [(g.x, g.y) for g in linestring.geoms]
                        routes.append(line_points)
                        route_points.update(line_points)
                        # TEMPORARY LIMITS
                        if len(routes) > 300:
                            break
                        count_paths += 1
                        count_path_points += len(linestring.geoms)
                else:
                    print("Unexpected type of geometry: %r" % geometry)
    print("  %d points, %d paths with %d total points" % (count_points, count_paths, count_path_points))

all_points = points.union(route_points)
print("%d unique points including routes" % len(all_points))

Name: taxi_routes; format: kml; length: 24602482
  0 points, 300 paths with 119939 total points
Name: fire_stations; format: kml; length: 10200
  30 points, 0 paths with 0 total points
74486 unique points including routes


## Generating time and location data

Let's generate a sample set of simulated moving cellphone data

In [2]:
class gen_params:
    PHONES = 5000
    MINUTES = 60*24*14
    SAMPLE_FREQUENCY = 7
    SAMPLE_STDEV = 2
    LOCATIONS_AVG = 2
    LOCATIONS_STDEV = 4
    TRIPS_PER_DAY_AVG = 1
    TRIPS_PER_DAY_STDEV = 1.5
    TRIP_TIME_STDEV = SAMPLE_FREQUENCY*10
    TRIP_STDEV = 10/111111. # about 10 meters
    WALK_STDEV = 2/111111. # about 2 meters
    LOCATION_STDEV = 1000/111111. # about 1 km

time_phone_location = []

total_trips = 0
total_locations = 0

min_x, max_x, min_y, max_y = None, None, None, None

def gen_data():
    global total_trips
    global total_locations
    for phone_id in range(gen_params.PHONES):
        trips_per_day = max(random.gauss(gen_params.TRIPS_PER_DAY_AVG, gen_params.TRIPS_PER_DAY_STDEV), 0.1)
        location_count = max(int(random.gauss(gen_params.LOCATIONS_AVG, gen_params.LOCATIONS_STDEV)), 1)
        location_bases = random.sample(points, location_count)
        locations = [(x+random.gauss(0, gen_params.LOCATION_STDEV), y+random.gauss(0, gen_params.LOCATION_STDEV)) for (x, y) in location_bases]
        minute = 0.0
        location_n = random.randint(0, location_count-1)
        total_locations += location_count
        x, y = locations[location_n]
        trips_taken = 0
        in_trip = False
        while minute < gen_params.MINUTES:
            minute += random.gauss(gen_params.SAMPLE_FREQUENCY, gen_params.SAMPLE_STDEV)
            if in_trip:
                if trip_point >= len(trip):
                    in_trip = False
                    # get a different location to the last one
                    if location_count > 1:
                        new_location_n = random.randint(0, location_count-2)
                        location_n = new_location_n + 1 if new_location_n >= location_n else new_location_n
                    else:
                        location_n = 0
                    x, y = locations[location_n]
                else:
                    x, y = trip[trip_point]
                    x += random.gauss(0, gen_params.TRIP_STDEV)
                    y += random.gauss(0, gen_params.TRIP_STDEV)
                    trip_point += 1
            if not in_trip:
                x += random.gauss(0, gen_params.WALK_STDEV)
                y += random.gauss(0, gen_params.WALK_STDEV)
                if (minute / 60*24) > (1/trips_per_day - trips_taken) + random.gauss(0, gen_params.TRIP_TIME_STDEV):
                    in_trip, trip, trip_point = True, random.choice(routes), 0
                    trips_taken += 1
                    total_trips += 1
            yield (minute, phone_id, (x, y))

tpl_series_cache_filename = make_cache_name('time-phone-location', gen_params, 'csv')
if not os.path.exists(tpl_series_cache_filename):
    print("Generating sample tracking data")
    with open(tpl_series_cache_filename, 'w') as f:
        series_writer = csv.writer(f)
        records = 0
        for minute, phone_id, (x, y) in gen_data():
            series_writer.writerow((str(minute), str(phone_id), str(x), str(y)))
            records += 1
        print("Tracked %d phones and generated %d time-phone-location tuples, with %d trips between %d phone-locations" % (gen_params.PHONES, records, total_trips, total_locations))


Generating sample tracking data
Tracked 5000 phones and generated 14403332 time-phone-location tuples, with 41286 trips between 14440 phone-locations


## Sectoring and Anonymizing Time-Phone-Location Data

We divide the location set into roughly square sectors.

We then generate a sector ID for each location in the time-phone-location dataset

Finally we store a hash of the time and sector ID, rather than the location, whilst still remembering sector adjacency for that time

In [5]:
class grid_params:
    GRID_SIZE = 25/111111.  # a rough estimate
    TIME_POCKET = 60 # minutes

def distance(origin, destination):
    # thanks https://gist.github.com/rochacbruno/2883505
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km
    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c
    return d

def coords_to_grid(x, y):
    return int(x / grid_params.GRID_SIZE), int(y / grid_params.GRID_SIZE)

min_x, max_x, min_y, max_y = None, None, None, None

unique_track_points = set()
unique_grid_points = set()

def gen_grid():
    global min_x, max_x, min_y, max_y
    with open(tpl_series_cache_filename) as csv_file:
        series_reader = csv.reader(csv_file)
        for row in series_reader:
            minute_s, phone_id_s, x_s, y_s = row
            minute, phone_id, x, y = float(minute_s), int(phone_id_s), float(x_s), float(y_s)
            unique_track_points.add((x, y))
            if min_x is None or x < min_x:
                min_x = x
            if max_x is None or x > max_x:
                max_x = x
            if min_y is None or y < min_y:
                min_y = y
            if max_y is None or y > max_y:
                max_y = y
            yield (minute, phone_id, coords_to_grid(x, y))

tpg_series_cache_filename = make_cache_name('time-phone-grid', grid_params, 'csv')
if not os.path.exists(tpg_series_cache_filename):
    print ("Converting to grid")
    with open(tpg_series_cache_filename, 'w') as f:
        series_writer = csv.writer(f)
        for minute, phone_id, (gx, gy) in gen_grid():
            unique_grid_points.add((gx, gy))
            series_writer.writerow((str(minute), str(phone_id), str(gx), str(gy)))
    mid_x, mid_y = (min_x + max_x)/2, (min_y + max_y)/2
    x_distance = distance((min_x, mid_y), (max_x, mid_y))
    y_distance = distance((mid_x, min_y), (mid_x, max_y))
    print("The X values lie between %r and %r - a distance of %r km" % (min_x, max_x, x_distance))
    print("The Y values lie between %r and %r - a distance of %r km" % (min_y, max_y, y_distance))

    print("%d unique tracked points reduce to %d unique grid points" % (len(unique_track_points), len(unique_grid_points)))

def time_grid_to_bucket(minute, gx, gy):
    hour = int(minute / grid_params.TIME_POCKET)
    return hashlib.sha1((b'%d:%d,%d' % (hour, gx, gy))).hexdigest()

def gen_buckets():
    with open(tpg_series_cache_filename, 'r') as csv_file:
        series_reader = csv.reader(csv_file)
        for row in series_reader:
            minute_s, phone_id_s, gx_s, gy_s = row
            minute, phone_id, gx, gy = float(minute_s), int(phone_id_s), float(gx_s), float(gy_s)
            yield (phone_id, time_grid_to_bucket(minute, gx, gy))

pb_series_cache_filename = make_cache_name('phone-bucket', grid_params, 'csv')
if not os.path.exists(pb_series_cache_filename):
    print ("Converting to buckets")
    with open(pb_series_cache_filename, 'w') as f:
        series_writer = csv.writer(f)
        for phone_id, bucket in gen_buckets():
            series_writer.writerow((str(phone_id), str(bucket)))

# TODO: make these file-based too
phone_bucket_map = {}
bucket_phone_map = {}

def populate_bucket_maps():
    print("Populating phone<->bucket maps")
    with open(pb_series_cache_filename, 'r') as csv_file:
        series_reader = csv.reader(csv_file)
        for row in series_reader:
            phone_id_s, bucket = row
            phone_id = int(phone_id_s)
            phone_bucket_map.setdefault(phone_id, set()).add(bucket)
            bucket_phone_map.setdefault(bucket, set()).add(phone_id)

populate_bucket_maps()
print("%d phones map to %d buckets" % (len(phone_bucket_map), len(bucket_phone_map)))

Populating phone<->bucket maps
5000 phones map to 6195196 buckets


## Querying the Buckets for a particular phone

We now illustrate how having identified a particular phone, we can search for phones with proximity to that phone in the covered period

This algorithm needs to be extended to also search adjacent buckets (which need to be recorded)

In [6]:
bucket_result_sizes = []

for search_num in range(5):
    search_phone = random.randint(0, gen_params.PHONES)

    print("Searching for phone ID %d" % search_phone)

    buckets = phone_bucket_map[search_phone]

    target_phones = set()

    print("  Found %d buckets" % len(buckets))

    for bucket in buckets:
        bucket_phones = bucket_phone_map.get(bucket, [])
        bucket_result_sizes.append(len(bucket_phones))
        target_phones.update(bucket_phones)

    print("  Found %d phones" % len(target_phones))
    
bucket_len_dist = {size: bucket_result_sizes.count(size) for size in set(bucket_result_sizes)}
print("Bucket result sizes:")
import pprint
pprint.pprint(bucket_len_dist)

Searching for phone ID 3482
  Found 2116 buckets
  Found 941 phones
Searching for phone ID 2824
  Found 2256 buckets
  Found 829 phones
Searching for phone ID 3939
  Found 2008 buckets
  Found 1368 phones
Searching for phone ID 1929
  Found 2313 buckets
  Found 542 phones
Searching for phone ID 953
  Found 2241 buckets
  Found 973 phones
Bucket result sizes:
{1: 4236,
 2: 2032,
 3: 1241,
 4: 808,
 5: 520,
 6: 364,
 7: 224,
 8: 169,
 9: 109,
 10: 70,
 11: 69,
 12: 42,
 13: 34,
 14: 32,
 15: 30,
 16: 49,
 17: 28,
 18: 31,
 19: 18,
 20: 26,
 21: 22,
 22: 18,
 23: 7,
 24: 13,
 25: 9,
 26: 2,
 27: 4,
 28: 9,
 29: 8,
 30: 8,
 31: 6,
 32: 12,
 33: 8,
 34: 6,
 35: 12,
 36: 11,
 37: 6,
 38: 4,
 39: 10,
 40: 7,
 41: 10,
 42: 11,
 43: 4,
 44: 5,
 45: 5,
 46: 5,
 47: 6,
 48: 6,
 49: 11,
 50: 11,
 51: 7,
 52: 7,
 53: 6,
 54: 9,
 55: 8,
 56: 13,
 57: 16,
 58: 4,
 59: 6,
 60: 8,
 61: 7,
 62: 7,
 63: 11,
 64: 11,
 65: 7,
 66: 7,
 67: 7,
 68: 8,
 69: 9,
 70: 6,
 71: 7,
 72: 6,
 73: 12,
 74: 6,
 75: 8,
